In [216]:
# built-in imports
import re
from json import dump
import pandas as pd
import re
from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

import json
import pandas as pd
import numpy as np
import re

In [217]:
# read in the original json file
'''!!!!!!!!!!!!! Note the file position and file name !!!!!!!!!!!!!!'''
path = "postcode3000.json"
with open(path, "r") as f:
    property_data_postcode = json.load(f)

In [218]:
property_data_postcode

{'https://www.domain.com.au/2108-288-spencer-street-melbourne-vic-3000-15572564': {'name': '2108/288 Spencer Street Melbourne VIC 3000',
  'cost_text': '$350',
  'bed_num': 1,
  'bath_num': 1,
  'park_num': 0,
  'type': 'Apartment / Unit / Flat',
  'furnitured': 'Yes',
  'pool': 'Yes',
  'gym': 'No',
  'coordinates': [-37.813775, 144.9520948],
  'desc': 'Kim Te',
  'features': '1 Bed1 Bath− Parking'},
 'https://www.domain.com.au/1505-250-elizabeth-street-melbourne-vic-3000-16078181': {'name': '1505/250 Elizabeth Street Melbourne VIC 3000',
  'cost_text': '$520.00 per week',
  'bed_num': 2,
  'bath_num': 1,
  'park_num': 0,
  'type': 'Apartment / Unit / Flat',
  'furnitured': 'No',
  'pool': 'No',
  'gym': 'No',
  'coordinates': [-37.812564, 144.9628184],
  'desc': 'Barbara Baltas',
  'features': '2 Beds1 Bath− Parking'},
 'https://www.domain.com.au/2413-220-spencer-street-melbourne-vic-3000-16046123': {'name': '2413/220 Spencer Street Melbourne VIC 3000',
  'cost_text': '$450 per week'

In [219]:
# build dataframe for property data looped with postcode
property_df = pd.DataFrame()

rent = []
address =[]
features = []
type = []


pool = []
gym = []
furnitured = []

bed_num = []
bath_num = []
park_num = []


latitude = []
longitude = []
coordinates = []


# lopp all the data in json file
for i in property_data_postcode:
    # print(i)

    # append all the useful features in dataframe
    address.append(property_data_postcode[i]["name"])
    rent.append(property_data_postcode[i]["cost_text"])
    features. append(property_data_postcode[i]["features"])
    type.append(property_data_postcode[i]["type"])
    
    pool.append(property_data_postcode[i]["pool"])
    gym.append(property_data_postcode[i]["gym"])
    furnitured.append(property_data_postcode[i]["furnitured"])

    # print(property_data_postcode[i]["bed_num"])
    # bed_num.append(property_data_postcode[i]["bed_num"])
    # bath_num.append(property_data_postcode[i]["bath_num"])
    # park_num.append(property_data_postcode[i]["park_num"])
    # coordinates.append(property_data_postcode[i]["coordinates"])


    latitude.append(property_data_postcode[i]["coordinates"][0])
    longitude.append(property_data_postcode[i]["coordinates"][1])


# build dataframe
property_df["address"] = address
property_df["rent"] = rent
property_df["features"] = features
property_df["type"] = type
property_df["furnitured"] = furnitured

# property_df["bed_num"] = bed_num
# property_df["bath_num"] = bath_num
# property_df["park_num"] = park_num

property_df["gym"] = gym
property_df["pool"] = pool
# property_df["coordinates"] = coordinates
property_df["latitude"] = latitude
property_df["longitude"] = longitude


# have a look of dataframe
property_df.head()

,address,rent,features,type,furnitured,gym,pool,latitude,longitude
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,Yes,-37.813775,144.952095
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,Yes,-37.815716,144.952956
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303


In [220]:
# check duplicate
property_df.shape

(15459, 9)

In [221]:
# check duplicate
property_df.duplicated().sum()

114

In [222]:
# drop duplicate
unique_property_df = property_df.drop_duplicates(keep="first")
unique_property_df.shape

(15345, 9)

In [223]:
# check
15459-15345

114

In [224]:
# reset the index of unique property data frame
unique_property_df = unique_property_df.reset_index(drop = True)
unique_property_df

,address,rent,features,type,furnitured,gym,pool,latitude,longitude
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,Yes,-37.813775,144.952095
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,Yes,-37.815716,144.952956
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303
...,...,...,...,...,...,...,...,...,...
15340,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,4 Beds2 Baths− Parking,House,Yes,No,No,-38.641250,145.703128
15341,14 Inverloch Parade Inverloch VIC 3996,$420 wk,2 Beds1 Bath− Parking,House,No,No,No,-38.628714,145.735808
15342,10 Hopetoun Street Inverloch VIC 3996,$370 wk,2 Beds1 Bath1 Parking,House,Yes,No,No,-38.634124,145.730686
15343,Inverloch VIC 3996,$475 wk,3 Beds2 Baths1 Parking,House,No,No,No,-38.631461,145.729364


In [225]:
# get postcode for all property
postcode=[]
for i in range(len(unique_property_df)):
    postcode.append(int(unique_property_df["address"][i][-4:]))

print("The number of postcode is: ", len(postcode))

The number of postcode is:  15345


In [226]:
# add postcode in unique property data frame
unique_property_df["postcode"] = postcode

In [227]:
# save a copy of unique property data frame
unique_property_df.to_csv("unique property data.csv", index = None)

In [228]:
# merge SA2 code and postcode
df_postcode = pd.read_csv('australian_postcodes.csv')
vic_postcode = df_postcode[df_postcode['state'] == 'VIC']
vic_postcode = vic_postcode.loc[:, ['postcode', 'SA2_MAINCODE_2016', 'SA2_NAME_2016']].reset_index(drop=True)
result = pd.merge(unique_property_df, vic_postcode, how='outer', sort=True, on='postcode')
result.head()

,address,rent,features,type,furnitured,gym,pool,latitude,longitude,postcode,SA2_MAINCODE_2016,SA2_NAME_2016
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,Yes,-37.813775,144.952095,3000,206041122.0,Melbourne
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818,3000,206041122.0,Melbourne
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,Yes,-37.815716,144.952956,3000,206041122.0,Melbourne
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265,3000,206041122.0,Melbourne
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303,3000,206041122.0,Melbourne


In [229]:
result.shape

(67890, 12)

In [230]:
result.duplicated().sum()

35249

In [231]:
result.drop_duplicates(keep="first")

,address,rent,features,type,furnitured,gym,pool,latitude,longitude,postcode,SA2_MAINCODE_2016,SA2_NAME_2016
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,Yes,-37.813775,144.952095,3000,206041122.0,Melbourne
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818,3000,206041122.0,Melbourne
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,Yes,-37.815716,144.952956,3000,206041122.0,Melbourne
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265,3000,206041122.0,Melbourne
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303,3000,206041122.0,Melbourne
...,...,...,...,...,...,...,...,...,...,...,...,...
67885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8111,206041122.0,Melbourne
67886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8120,206041122.0,Melbourne
67887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8205,206041122.0,Melbourne
67888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8785,212041311.0,Dandenong


In [232]:
# clean the rent in rent column in unique property dataframe
rent_df_new = pd.DataFrame()
rent_df_new["rent"] = unique_property_df["rent"]

for i in range(len(rent_df_new["rent"])):
    # print(i)
    price = rent_df_new["rent"][i].lower()
    if ("p.a." in price) or ("annually" in price) or ("pa." in price):
        # print(i)
        # print(price)
        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/52
        # print(float(rent[0].replace(",", "")))

    # two weeks pay /2
    elif "fortnightly" in price:
        # print(price)
        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))/2

    # per night *7
    elif "night" in price:
        # print(price)
        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        # print(rent)
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
        
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))*7
        # print(rent_df_new["rent"][i])

    elif ("pcm" in price) and ("pw" in price):
        # print(price)
        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        # print(rent)
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))

    elif ("month" in price):
        # print(price)
        # print(rent)
        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
        rent_df_new["rent"][i] = float(rent[0].replace(",", ""))
        
    else:

        rent = re.findall(r"[$](\d+\.?,?\d*)", price)
        if len(rent) == 0:
            rent = re.findall(r"(\d+\.?,?\d*)", price)
            if len(rent) == 0:
                rent_df_new["rent"][i] = 0
            else:
                rent_df_new["rent"][i] = float(rent[0].replace(",", ""))
        elif len(rent) == 2:
            rent = float(rent[0].replace(",", "")) + float(rent[1].replace(",", ""))
            rent_df_new["rent"][i] = rent/2
        else:
            rent_df_new["rent"][i] = float(rent[0].replace(",", ""))


# save two decimal places
formater="{0:.02f}".format
rent_df_new = rent_df_new.applymap(formater)

# save a copy of rent cleaned csv
rent_df_new.to_csv("cleaned_rent.csv", index = None)


In [233]:
# check if cleaned
rent_perweek = []
for i in rent_df_new["rent"]:
    rent_perweek.append(i)
rent_perweek

['350.00',
 '520.00',
 '450.00',
 '550.00',
 '550.00',
 '900.00',
 '680.00',
 '360.00',
 '400.00',
 '400.00',
 '575.00',
 '1800.00',
 '480.00',
 '420.00',
 '600.00',
 '600.00',
 '550.00',
 '440.00',
 '0.00',
 '0.00',
 '440.00',
 '320.00',
 '395.00',
 '370.00',
 '2500.00',
 '550.00',
 '680.00',
 '550.00',
 '430.00',
 '420.00',
 '410.00',
 '950.00',
 '70.00',
 '300.00',
 '550.00',
 '700.00',
 '530.00',
 '350.00',
 '650.00',
 '380.00',
 '500.00',
 '595.00',
 '550.00',
 '695.00',
 '250.00',
 '600.00',
 '200.00',
 '400.00',
 '650.00',
 '610.00',
 '500.00',
 '695.00',
 '680.00',
 '380.00',
 '580.00',
 '235.00',
 '550.00',
 '350.00',
 '600.00',
 '400.00',
 '1100.00',
 '370.00',
 '2800.00',
 '4500.00',
 '440.00',
 '440.00',
 '389.00',
 '700.00',
 '700.00',
 '720.00',
 '680.00',
 '400.00',
 '480.00',
 '500.00',
 '260.00',
 '400.00',
 '620.00',
 '395.00',
 '650.00',
 '580.00',
 '670.00',
 '355.00',
 '550.00',
 '650.00',
 '780.00',
 '620.00',
 '450.00',
 '400.00',
 '0.00',
 '650.00',
 '450.00',
 

In [234]:
# add cleaned amount of rent per week in unique property data frame
unique_property_df["rent_perweek"] = rent_df_new["rent"]
unique_property_df.head()

,address,rent,features,type,furnitured,gym,pool,latitude,longitude,postcode,rent_perweek
0,2108/288 Spencer Street Melbourne VIC 3000,$350,1 Bed1 Bath− Parking,Apartment / Unit / Flat,Yes,No,Yes,-37.813775,144.952095,3000,350.00
1,1505/250 Elizabeth Street Melbourne VIC 3000,$520.00 per week,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.812564,144.962818,3000,520.00
2,2413/220 Spencer Street Melbourne VIC 3000,$450 per week,2 Beds2 Baths1 Parking,Apartment / Unit / Flat,No,No,Yes,-37.815716,144.952956,3000,450.00
3,2205/22-24 Jane Bell Lane Melbourne VIC 3000,$550,2 Beds1 Bath− Parking,Apartment / Unit / Flat,No,No,No,-37.810220,144.966265,3000,550.00
4,G11/61 MacKenzie Street Melbourne VIC 3000,$550.00 per week,2 Beds1 Bath1 Parking,Apartment / Unit / Flat,Yes,No,No,-37.807871,144.966303,3000,550.00


In [235]:
# save a copy of cleaned unique property data csv
unique_property_df.to_csv("cleaned_rent_unique_property_data.csv", index = None)

In [236]:
# unique_property_df["features"].to_csv("check.csv")

In [237]:
unique_property_df.shape

(15345, 11)

In [238]:
feature_df = pd.DataFrame()
feature_df["features"] = unique_property_df["features"]

bed_num = []
for i in range(len(feature_df["features"])):
    bed_num.append(0)
bath_num = []
for i in range(len(feature_df["features"])):
    bath_num.append(0)
car_park_num = []
for i in range(len(feature_df["features"])):
    car_park_num.append(0)

feature_df["bed_num"] = bed_num
feature_df["bath_num"] = bath_num
feature_df["car_park_num"] = car_park_num

In [239]:
for i in range(len(feature_df["features"])):
    # print(feature_df["features"][i])
    if feature_df["features"][i] == "":
        # print("yes")
        feature_df["bed_num"][i] = 0
        feature_df["bath_num"][i] = 0
        feature_df["car_park_num"][i]= 0
    elif feature_df["features"][i].split(" ")[0] == "−" and feature_df["features"][i].split(" ")[3] != "0":
        # print(feature_df["features"][i].split(" "))
        feature_df["bed_num"][i] = feature_df["features"][i].split(" ")[1]
        feature_df["bath_num"][i] = feature_df["features"][i].split(" ")[2][-1]
        feature_df["car_park_num"][i]= feature_df["features"][i].split(" ")[3][-1]
    elif feature_df["features"][i].split(" ")[0] == "−" and feature_df["features"][i].split(" ")[3] == "0":
        # print(feature_df["features"][i].split(" "))
        feature_df["bed_num"][i] = feature_df["features"][i].split(" ")[1]
        feature_df["bath_num"][i] = feature_df["features"][i].split(" ")[3]
        feature_df["car_park_num"][i]= feature_df["features"][i].split(" ")[4][-1]
    else:
        if len(feature_df["features"][i].split(" ")[1]) == 0:
            # print("yes")
            # print(feature_df["features"][i].split(" "))
            feature_df["bed_num"][i] = "Not Clear"
            feature_df["bath_num"][i] = "Not Clear"
            feature_df["car_park_num"][i]= "Not Clear"
        else:
            feature_df["bed_num"][i] = feature_df["features"][i].split(" ")[0]
            feature_df["bath_num"][i] = feature_df["features"][i].split(" ")[1][-1]
            feature_df["car_park_num"][i]= feature_df["features"][i].split(" ")[2][-1]   

<ipython-input-239-4b86d8e1bf71>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["bed_num"][i] = feature_df["features"][i].split(" ")[0]
<ipython-input-239-4b86d8e1bf71>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["bath_num"][i] = feature_df["features"][i].split(" ")[1][-1]
<ipython-input-239-4b86d8e1bf71>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df["car_park_num"][i]= feature_df["features

In [240]:
# save a copy of cleaned features
feature_df.reset_index(drop=True)
feature_df.to_csv("check_features.csv", index = None)

In [241]:
# put number of bed, number of bath and number of car park in property dataframe
unique_property_df["bed_num"] = feature_df["bed_num"]
unique_property_df["bath_num"] = feature_df["bath_num"]
unique_property_df["car_park_num"] = feature_df["car_park_num"]

In [242]:
# save a copy of cleaned feature property dataframe
unique_property_df.head()
unique_property_df.to_csv("cleaned_feature_property_data.csv", index = None)